In [1]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [2]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained('try_new/Pegasus/Tokenizer/')

f:\paraphrase\paraphraseenv\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
model = PegasusForConditionalGeneration.from_pretrained('try_new/Pegasus/Model/').to(torch_device)

In [4]:
device = torch_device
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

In [37]:
train_encodings_1 = tokenizer.batch_encode_plus(pd.read_csv('ms_all.csv')[['#1 String']].values[:,0], truncation=True, padding=True)
train_encodings_2 = tokenizer.batch_encode_plus(pd.read_csv('ms_all.csv')[['#2 String']].values[:,0], truncation=True, padding=True)

In [49]:
import torch

class MSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings_1, encodings_2):
        self.inputs = train_encodings_1
        self.targets = train_encodings_2

    def __getitem__(self, idx):
        input = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        target = {'decoder_'+str(key): torch.tensor(val[idx]) for key, val in self.targets.items()}
        input["labels"] = target["decoder_input_ids"]
        return {**input, **target}

    def __len__(self):
        return len(self.inputs['input_ids'])

In [50]:
train_dataset = MSDataset(train_encodings_1, train_encodings_2)

In [51]:
train_dataset[0]

{'input_ids': tensor([66860, 62954,  6462,   169,  2936,   108,  2901,   178,   568,   198,
           544,  5945,   194,   108,   113, 14981, 75538,   169,  1812,   107,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([ 2014, 45593,   112,   342,   130,   209,   198,   544,  5945,   194,
           108, 66860, 62954,  6462,   169,  2936,   113, 14981, 75538,   169,
          1812,   107,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]),
 'decoder_input_ids

In [52]:
from transformers import Trainer, TrainingArguments

In [53]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

In [54]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=train_dataset            # evaluation dataset
)

In [55]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 